In [ ]:
# Installation de lightweight_mmm
# Doc de LightWeight : https://lightweight-mmm.readthedocs.io/en/latest/models.html
# Github : https://github.com/google/lightweight_mmm/
!pip install --upgrade git+https://github.com/google/lightweight_mmm.git
!pip install numpyro==0.13.2 ##

  Cloning https://github.com/google/lightweight_mmm.git to /tmp/pip-req-build-ujm09sk9
  Running command git clone --filter=blob:none --quiet https://github.com/google/lightweight_mmm.git /tmp/pip-req-build-ujm09sk9
  Resolved https://github.com/google/lightweight_mmm.git to commit 4406aaa77bddc5b0d73d31c6cf4f2ace03f3ffda
  Preparing metadata (setup.py) ... done


In [ ]:
##################################################
# LISTE DES VARIABLES QUI PEUVENT ETRE MODIFIEES #
##################################################

# URL de la gSheet contenant les données
url = 'https://docs.google.com/spreadsheets/d/1_z2JvoIjHHOPLWILqa2vugPXDootQdTgmm2_gkj6Nq8'

# URL de la gSheet pour enregistrer les résultats
url_results = 'https://docs.google.com/spreadsheets/d/1-WfTXrYOD-DnAw5uZbz8AmBNcOzUgyQLnRtiWVmlQMs/edit#gid=1399420582'

# URL de la gSheet pour enregistrer les raw data
url_raw = 'https://docs.google.com/spreadsheets/d/1-w1aA3N2zl6Hz2MrOrMwx2viGkjP7RBEPtlVZrApfGw/edit#gid=219060037'

# Nom de l'onglet de la gSheet ou sont les données (bien respecter la casse)
sheet_name = 'Weekly'

#Estimation initiale du % des resultats attribuables à l'organique (entre 0 et 1)
organic_percent_prior = 0

#Pourcentage des données dans le training set (entre 0 et 1, idealement 0.9)
train_percent = 0.9

# Possibilités pour le modele de saturation/lag : carryover, hill_adstock ou adstock
model_name = 'carryover'

# Granularité des données : daily ou weekly
granularity = 'weekly'

# Saisonnalité 'jours de la semaine' (uniquement pour des données daily, True ou False sans guillemets)
weekday_seasonality = False

# Pour pouvoir tester l'influence des custom priors sur les resultats, on peut par exemple jouer sur le paramètre de saturation
# Ca ne fonctionnera que pour le modèle Adstock ou Carryover. Le paramètre suite une loi de probabilité Beta(1,saturation).
# Plus la valeur est élevée, moins il y a de saturation
# La valeur de base est 9. Elle doit etre supérieure à 1.
saturation = 1

# Paramètres d'entrainement du modèle (influence sur la durée d'entrainement et la convergence)
# Nombre de cycles d'entrainement rejetés initialement (valeur de base : 1000)
# Ici on met de petites valeurs pour avoir une v1 qui tourne vite
number_warmup=200
# Nombre de cycles d'entrainement acceptés (valeur de base : 1000)
number_samples=200
# Nombre de chaines de Markov (nombre d'entrainements différents, valeur de base : 2)
number_chains=2
# 'Graine' pour la reproductibilité (un entrainement avec un même nombre menera au même resultat)
SEED = 99

# Paramètres d'optimisation
# Nb de jours ou Nb de semaines sur lesquels on va chercher à optimiser le mix media
n_time_periods = 10
# Pourcentage maximum de baisse par rapport au budget précédent de chaque network pour les recommandations (valeur de base 0.2)
bounds_lower_pct = 0.2
# Pourcentage maximum d'augmentation par rapport au budget précédent de cahque network pour les recommandations (valeur de base 0.2)
bounds_upper_pct = 0.2

In [ ]:
##################################################
# 1- INITIALISATION ET PREPARATION DES DONNEES   #
##################################################

import pandas as pd
import gspread
import numpy as np
from google.colab import auth
from google.auth import default
import jax.numpy as jnp
import jax
import numpyro
import arviz
from lightweight_mmm import models
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils
from sklearn import metrics
from sklearn.metrics import r2_score
import scipy.stats as st

# Fonction utilisée pour enregistrer les résultats sous forme de gSheet
def update_sheet(spreadsheet, sheet_name, dataframe):
    # Vérifier si l'onglet existe
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.WorksheetNotFound:
        # Créer l'onglet si celui-ci n'existe pas
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="1", cols="1")
    # Effacer le contenu existant et mettre à jour avec le nouveau dataframe
    worksheet.clear()
    worksheet.update([dataframe.reset_index().columns.values.tolist()] + dataframe.reset_index().values.tolist())

# Ouverture de la gSheet comme pandas dataframe
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
spreadsheet = gc.open_by_url(url)
worksheet = spreadsheet.worksheet(sheet_name)
data = worksheet.get_all_values()
df = pd.DataFrame(data)

# Ouverture de la gSheet pour enregistrer les résultats
spreadsheet_results = gc.open_by_url(url_results)
spreadsheet_raw = gc.open_by_url(url_raw)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,,Ventes,Media,Media,Media,Media,Media,Media,Media,Media,...,Media,Media,Media,Media,Media,Media,Media,Media,Media,Media
1,,Chiffre d'Affaires,Print_Others,Print_ELLE,Print_MADAME FIGARO,Print_MARIE CLAIRE,Print_VERSION FEMINA,Print_GALA,Print_COSMOPOLITAN,Print_FEMME ACTUELLE,...,TV_TF1 SERIES FILMS,TV_NRJ12,Search,Social,Display,OOH_JC DECAUX VITRINE 2M2 NATIONAL,OOH_METROBUS METRO NATIONAL,OOH_CLEAR CHANNEL SHOPPING MALLS,OOH_MEDIAGARES TRANSPORT,OOH_Others
2,Date,Budget,€,€,€,€,€,€,€,€,...,€,€,€,€,€,€,€,€,€,€
3,22/06/2020,1 155 905,0,9 405,19 228,15 257,0,0,14 526,0,...,0,0,4 513,6 859,0,0,0,0,0,0
4,29/06/2020,1 790 591,0,9 050,0,0,0,7 817,0,11 767,...,0,0,7 591,11 039,0,0,0,15 020,0,1 499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,22/05/2023,2 135 932,27 247,0,0,25 366,0,0,22 660,0,...,0,0,10 500,2 000,0,0,0,0,0,0
156,29/05/2023,1 863 808,10 780,0,0,0,0,10 472,0,0,...,0,0,10 500,3 000,0,0,0,0,0,0
157,05/06/2023,2 171 948,0,9 900,12 540,0,0,7 392,0,0,...,0,0,10 500,7 000,0,60 768,0,0,0,3 688
158,12/06/2023,1 977 824,14 080,9 900,0,0,0,0,0,0,...,0,0,10 500,3 000,0,24 302,0,0,0,1 474


In [ ]:
#Cleaning des données
df.columns = df.iloc[0] + '_' + df.iloc[1]# Définir la première ligne comme en-tête (basé sur les 2 premières lignes de chaque col)
if not df.iloc[2].empty: # Vérifier si la troisième ligne n'est pas vide
    df.columns = df.columns + '_' + df.iloc[2] # Ajouter le contenu de la troisième ligne à la fin des noms de colonnes
df.columns.values[0] = 'Date' # On renomme la première colonne
df.columns.values[1] = 'Ventes' # On renomme la deuxième colonne
df = df.iloc[3:].reset_index(drop=True) # Enlever les 3 premières lignes
# Convertir toutes les autres colonnes en float
for col in df.columns[1:]:
    df[col] = df[col].replace(r'\s+', '', regex=True) #suppression des espaces
    df[col] = pd.to_numeric(df[col], errors='coerce')
#On remplace les cellules vides par des 0
df.fillna(0, inplace=True)

# Liste des networks = on sélectionne les variables qui commencent par Media, et on leur enlève le '_€' à la fin
networks_list = [col.replace('_€', '') for col in df.filter(like='Media').filter(like='_€').columns]
# Liste des extra_features = les variables qui commencent par 'Extra'
extra_list = [col[:-1] for col in df.filter(like='Extra').columns]
# Liste des toutes les variables hors Dates et Ventes
variables_list = networks_list + extra_list

# Transformation des données en un format utilisable par LightWeight
# Les données d'entrée doivent etre sous forme de tableaux Numpy
media_data_budget = df[df.filter(like='Media').filter(like='€').columns].to_numpy() # Sélection des variables commençant par "Media" et finissant par "€"
costs = sum(media_data_budget) # C'est la somme des dépenses représenté par chaque média

# Definition des colonnes media à garder (impressions si disponible, sinon budget)
to_keep = [col for col in df.columns if col.startswith('Media') and (
    col.endswith('_€') and not any(other.startswith(col[:-2]) and not other.endswith('_€') for other in df.columns) or not col.endswith('_€'))]

media_data = df[to_keep].to_numpy()
target = df['Ventes'].to_numpy(dtype='float32')
extra_features = df[df.filter(like='Extra').columns].to_numpy() # Sélection des variables commençant par "Extra"

# Intégration data dans le fichier raw
update_sheet(spreadsheet_raw, 'DATA', df)

# INTEGRATION
# Pour l'integration backend, les étapes précédentes ne sont pas nécéssaires (il n'est pas nécéssaire de passer par une pandas dataframe)
# Il faudra juste fournir sous forme de tableau Numpy les éléments suivants: networks_list, media_data, target et costs ## et new_features aussi

In [ ]:
extra_list

[]

In [ ]:
##############################################################
# 2- SEPARATION TESTSET/TRAININGSET ET NORMAGE DES DONNEES   #
##############################################################

data_size = media_data.shape[0]
n_media_channels = media_data.shape[1]
split_point = round(train_percent*data_size)

# Media data
media_data_train = media_data[:split_point, ...]
media_data_test = media_data[split_point:, ...]
## Extra features
extra_features_train = extra_features[:split_point, ...] ##
extra_features_test = extra_features[split_point:, ...] ##
# Target
target_train = target[:split_point]

media_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
extra_features_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean) ##
target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)

cost_scaler = preprocessing.CustomScaler(multiply_by= (1-organic_percent_prior)/sum(costs) )
#Ce qu'on fait ici dans le scaler est clé pour le prior
#1- On divise par la somme des couts afin d'obtenir le % des couts par ce media
#2- On multiplie par le % du paid (1- le % de l'organique)
# dans le mmm.fit, media_prior représente l'estimation du % de contrib de chaque media sur les ventes.

media_data_train_scaled = media_scaler.fit_transform(media_data_train)
extra_features_train_scaled = extra_features_scaler.fit_transform(extra_features_train) ##
target_train_scaled = target_scaler.fit_transform(target_train)
costs_scaled = cost_scaler.fit_transform(costs)

In [ ]:
#CHANGED#
#Changement liste variables
#Todo : Faire les non-null sur toutes les variables

########################################
# 3- CHECK DE LA QUALITE DES DONNEES   #
########################################

# 3.1- Check du nombre de valeurs non nulles dans le training set
# Idealement il faut au moins 10 valeur non nulles par network, et jamais 0

def highlight_low_not_null(x: float,
                                  low_threshold: float=10) -> str:
    if x < low_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

res_df = pd.DataFrame((media_data_train != 0).sum(axis=0), index=networks_list, columns=['Non-Null Counts'])

update_sheet(spreadsheet_results, 'not_null_count', res_df)

res_df.style.format().applymap(highlight_low_not_null)

,Non-Null Counts
Media_Print_Others,87
Media_Print_ELLE,83
Media_Print_MADAME FIGARO,52
Media_Print_MARIE CLAIRE,30
Media_Print_VERSION FEMINA,26
Media_Print_GALA,68
Media_Print_COSMOPOLITAN,29
Media_Print_FEMME ACTUELLE,30
Media_Print_PARIS MATCH,24
Media_Print_BIBA,28


In [ ]:
#CHANGED#
# 3.2 On check qu'il n'y a pas de correlations trop fortes entre les spends des medias

correlations, variances, spend_fractions, variance_inflation_factors = preprocessing.check_data_quality(
    media_data=media_scaler.transform(media_data),
    target_data=target_scaler.transform(target),
    extra_features_data=extra_features_scaler.transform(extra_features), ##
    cost_data=costs_scaled)

correlations[0].index=variables_list+[correlations[0].index[-1]] ##
correlations[0].columns=variables_list+[correlations[0].columns[-1]] ##

update_sheet(spreadsheet_results, 'correlations', correlations[0])

correlations[0].style.background_gradient(cmap='RdBu', vmin=-1, vmax=1).format(precision=3)

,Media_Print_Others,Media_Print_ELLE,Media_Print_MADAME FIGARO,Media_Print_MARIE CLAIRE,Media_Print_VERSION FEMINA,Media_Print_GALA,Media_Print_COSMOPOLITAN,Media_Print_FEMME ACTUELLE,Media_Print_PARIS MATCH,Media_Print_BIBA,Media_Print_AVANTAGES,Media_Print_PSYCHOLOGIES,Media_TV_TF1,Media_TV_Others,Media_TV_M6,Media_TV_TMC,Media_TV_FRANCE 2,Media_TV_BFM TV,Media_TV_C8,Media_TV_TF1 SERIES FILMS,Media_TV_NRJ12,Media_Search,Media_Social,Media_Display,Media_OOH_JC DECAUX VITRINE 2M2 NATIONAL,Media_OOH_METROBUS METRO NATIONAL,Media_OOH_CLEAR CHANNEL SHOPPING MALLS,Media_OOH_MEDIAGARES TRANSPORT,Media_OOH_Others,target
Media_Print_Others,1.000,0.244,0.109,0.147,0.095,0.453,0.162,0.202,0.170,0.148,0.009,0.121,0.040,-0.002,-0.069,0.116,0.044,-0.015,0.071,-0.000,0.085,0.310,0.152,-0.100,-0.039,-0.059,-0.027,0.193,0.345,0.261
Media_Print_ELLE,0.244,1.000,0.087,0.023,0.127,0.431,-0.126,0.020,0.186,-0.002,0.003,0.145,0.229,0.242,0.225,0.307,0.215,0.294,0.307,0.307,0.305,0.207,0.189,0.196,-0.085,-0.072,0.086,-0.027,0.062,0.233
Media_Print_MADAME FIGARO,0.109,0.087,1.000,-0.023,0.094,0.029,-0.010,0.143,0.056,-0.169,-0.114,0.006,0.112,0.107,0.159,0.167,0.027,0.041,0.117,0.028,0.138,0.080,0.052,0.070,-0.056,-0.134,-0.034,0.052,0.023,0.014
Media_Print_MARIE CLAIRE,0.147,0.023,-0.023,1.000,-0.098,-0.102,0.474,-0.134,-0.044,0.333,0.321,-0.191,0.058,0.048,0.214,-0.012,0.013,0.008,0.066,0.095,0.084,0.071,0.113,0.140,0.001,0.045,0.027,-0.109,-0.065,0.229
Media_Print_VERSION FEMINA,0.095,0.127,0.094,-0.098,1.000,0.102,-0.145,0.204,0.343,-0.183,-0.063,-0.127,0.067,-0.013,-0.041,-0.090,0.102,0.054,-0.080,-0.072,-0.077,0.027,0.091,0.029,0.126,-0.102,0.381,-0.054,-0.007,0.243
Media_Print_GALA,0.453,0.431,0.029,-0.102,0.102,1.000,-0.110,0.266,0.132,-0.005,-0.037,0.238,0.019,0.073,-0.056,0.162,0.104,0.097,0.113,0.099,0.090,0.255,0.178,0.017,0.047,-0.104,0.180,0.268,0.420,0.120
Media_Print_COSMOPOLITAN,0.162,-0.126,-0.010,0.474,-0.145,-0.110,1.000,0.012,-0.084,0.484,0.357,-0.177,0.099,0.066,0.169,0.039,0.051,0.077,0.076,0.093,0.115,-0.038,0.131,0.069,-0.026,0.042,-0.074,-0.018,-0.061,0.224
Media_Print_FEMME ACTUELLE,0.202,0.020,0.143,-0.134,0.204,0.266,0.012,1.000,0.261,0.003,-0.030,-0.002,0.037,-0.091,-0.022,-0.096,-0.018,-0.063,-0.042,-0.056,-0.045,0.098,0.089,-0.000,-0.063,0.056,0.018,0.230,0.221,0.136
Media_Print_PARIS MATCH,0.170,0.186,0.056,-0.044,0.343,0.132,-0.084,0.261,1.000,-0.118,0.000,0.070,0.129,0.010,-0.100,-0.057,0.129,0.001,-0.079,-0.145,-0.134,0.154,0.100,-0.134,-0.054,0.020,0.090,0.021,-0.010,0.207
Media_Print_BIBA,0.148,-0.002,-0.169,0.333,-0.183,-0.005,0.484,0.003,-0.118,1.000,0.210,-0.172,-0.126,-0.034,-0.042,-0.002,0.101,-0.013,-0.072,0.084,0.009,-0.042,0.041,-0.089,-0.013,0.023,-0.098,0.041,-0.041,0.038


In [ ]:
# 3.3 Check des variances : il les faut idealement toutes inferieurs à 3

def highlight_variances(x: float,
                        low_variance_threshold: float=1.0e-3,
                        high_variance_threshold: float=3.0) -> str:

    if x < low_variance_threshold or x > high_variance_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

variances.index=variables_list ##
variances.columns=['Variance']

update_sheet(spreadsheet_results, 'variances', variances)

variances.style.format(precision=4).applymap(highlight_variances)

,Variance
Media_Print_Others,1.8719
Media_Print_ELLE,0.9790
Media_Print_MADAME FIGARO,2.1026
Media_Print_MARIE CLAIRE,3.9898
Media_Print_VERSION FEMINA,4.3836
Media_Print_GALA,1.3293
Media_Print_COSMOPOLITAN,4.7147
Media_Print_FEMME ACTUELLE,3.7065
Media_Print_PARIS MATCH,5.3824
Media_Print_BIBA,4.5970


In [ ]:
# 3.4 Importance de chaque network dans le mix media : on veut au moins 1% par media

def highlight_low_spend_fractions(x: float,
                                  low_spend_threshold: float=0.01) -> str:
    if x < low_spend_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

spend_fractions.index=networks_list

update_sheet(spreadsheet_results, 'spend_fractions', spend_fractions)

spend_fractions.style.format(precision=4).applymap(highlight_low_spend_fractions)

,fraction of spend
Media_Print_Others,0.0666
Media_Print_ELLE,0.0547
Media_Print_MADAME FIGARO,0.0425
Media_Print_MARIE CLAIRE,0.0336
Media_Print_VERSION FEMINA,0.0313
Media_Print_GALA,0.0296
Media_Print_COSMOPOLITAN,0.0239
Media_Print_FEMME ACTUELLE,0.0197
Media_Print_PARIS MATCH,0.0176
Media_Print_BIBA,0.0181


In [ ]:
# 3.5 Variance inflation factors : doit etre inferieur à 7

def highlight_high_vif_values(x: float,
                              high_vif_threshold: float=7.0) -> str:
    if x > high_vif_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

variance_inflation_factors.index=variables_list
variance_inflation_factors.columns=['Variance inflation factor']

update_sheet(spreadsheet_results, 'variance_inflation_factors', variance_inflation_factors)

variance_inflation_factors.style.format(precision=4).applymap(highlight_high_vif_values)


,Variance inflation factor
Media_Print_Others,2.0026
Media_Print_ELLE,1.7859
Media_Print_MADAME FIGARO,1.2769
Media_Print_MARIE CLAIRE,1.7930
Media_Print_VERSION FEMINA,1.9388
Media_Print_GALA,2.2434
Media_Print_COSMOPOLITAN,2.0041
Media_Print_FEMME ACTUELLE,1.5693
Media_Print_PARIS MATCH,1.6709
Media_Print_BIBA,2.2208


In [ ]:
###################################
# 4- ENTRAINEMENT DU MODELE MMM   #
###################################

#Définition du modèle MMM
mmm = lightweight_mmm.LightweightMMM(model_name=model_name)

In [ ]:
#CREATED#
#On ajoute la possibilité azu modélisateur des changer les custom
#Todo : proprifier et lister toutes les priors possibles en fonction des différents types de modèles

# CODE ICI FAIT A LA MAIN QU'IL FAUDRA AUTOMATISER (?)

#Ci-dessous on définit les custom priors pour un "hill_adstock". Ce ne sera pas les mêmes paramètres à mettre pour un "adstock" ou un "carryover"
channel_adstock_weights={'SEM&SEHM':0.01,
 'VOL':0.5,
 'TV':2}
adstock_lag_weight_priors=jnp.array(list(channel_adstock_weights.values()))

#Definition des custom priors
custom_priors= {"exponent":numpyro.distributions.Beta(concentration1=saturation, concentration0=1.)} if model_name != 'hill_adstock' else {}
# custom_priors = {
#                  "lag_weight": numpyro.distributions.Beta(concentration1=adstock_lag_weight_priors, concentration0=jnp.ones(len(adstock_lag_weight_priors),))
#                  }

In [ ]:
# Fit model
mmm.fit(
    media=media_data_train_scaled,
    media_names = networks_list,
    media_prior=costs_scaled,
    #extra_features=extra_features_train_scaled, ##
    target=target_train_scaled,
    number_warmup=number_warmup,
    number_samples=number_samples,
    weekday_seasonality=weekday_seasonality,
    seasonality_frequency = 365 if granularity == 'daily' else 52,
    number_chains = number_chains,
    custom_priors=custom_priors,
    seed=SEED)

# Enregistrement des raw data dans une gSheet
for key, np_array in mmm.trace.items():
    if np_array.ndim <= 2:
      df_raw = pd.DataFrame(np_array)
      update_sheet(spreadsheet_raw, key, df_raw)

# Résultats de l'entrainement
# The rule of thumb is that r_hat values for all parameters are less than 1.1
mmm.print_summary()

/usr/local/lib/python3.10/dist-packages/lightweight_mmm/lightweight_mmm.py:359: UserWarning: There are not enough devices to run parallel chains: expected 2 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(2)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = numpyro.infer.MCMC(
sample: 100%|██████████| 400/400 [15:07<00:00,  2.27s/it, 6 steps of size 1.28e-02. acc. prob=0.79] 



                                  mean       std    median      5.0%     95.0%     n_eff     r_hat
 ad_effect_retention_rate[0]      0.60      0.29      0.68      0.15      1.00    106.29      1.01
 ad_effect_retention_rate[1]      0.55      0.31      0.56      0.07      1.00    113.30      1.00
 ad_effect_retention_rate[2]      0.53      0.30      0.56      0.12      1.00    459.09      1.00
 ad_effect_retention_rate[3]      0.51      0.32      0.53      0.06      1.00     38.52      1.08
 ad_effect_retention_rate[4]      0.43      0.30      0.38      0.00      0.91     71.42      1.03
 ad_effect_retention_rate[5]      0.57      0.28      0.57      0.16      1.00    148.28      1.00
 ad_effect_retention_rate[6]      0.56      0.27      0.57      0.16      0.96    335.49      1.01
 ad_effect_retention_rate[7]      0.50      0.25      0.52      0.11      0.89    218.38      1.02
 ad_effect_retention_rate[8]      0.42      0.31      0.39      0.00      0.88    120.75      1.01
 ad_effec

In [ ]:
#CREATED#
#Permet de mieux visualiser les résultats du modèle
#Todo : Proprifier et faire des résultats tous faits en fonction des différents types de modèles


#Assembler un tableau résultats new_features + un tableau résultats media
#Séparer en fonction du typ de modèle:
#Si carryover : (ajouter "saturation")
#pd.DataFrame({'channels':networks_list, 'coeff':mmm.trace['coef_media'].mean(axis=0), 'adstock_decay':mmm.trace['ad_effect_retention_rate'].mean(axis=0)})
#Si hill_adstock ou adstock, ce sera d'autres KPIs à définir, et ajouter aide à la lecture pour chaque KPIs
pd.DataFrame({'channels':networks_list, 'coeff':mmm.trace['coef_media'].mean(axis=0), 'lag_weight':mmm.trace['lag_weight'].mean(axis=0), 'half_max_effective_concentration':mmm.trace['half_max_effective_concentration'].mean(axis=0)})

KeyError: 'lag_weight'

In [ ]:
#Résultats extra_features:
pd.DataFrame({'channels':extra_list, 'coeff':mmm.trace['coef_extra_features'].mean(axis=0)})

In [ ]:
# Densités de probabilité posterieure des coefficients médias (% des resultats attribués au média)
plot.plot_media_channel_posteriors(media_mix_model=mmm,channel_names=networks_list) ##

In [ ]:
#OPTIONNEL
#On met ici en commentaire car bcp d'affichage pour peu d'utilisation aujourd'hui

# Densités de probabilité prior et posterieure des coefficients médias (% des resultats attribués au média)
plot.plot_prior_and_posterior(media_mix_model=mmm)

In [ ]:
# Fit du modèle sur le training set (les données qui ont servi à l'entrainement)
plot.plot_model_fit(mmm, target_scaler=target_scaler)

In [ ]:
#NOT CHANGED, mais on a ajouté une librairie ancienne de numpy sinon ne fonctionne pas

# Avec mmm.predict on peut prédire les resultats pour des données nouvelles, à partir du modèle entrainé
new_predictions = mmm.predict(media=media_scaler.transform(media_data_test),
                              seed=SEED)

# Fit du modèle sur le test set (les données encore jamais vues)
plot.plot_out_of_sample_model_fit(out_of_sample_predictions=target_scaler.inverse_transform(new_predictions),
                                 out_of_sample_target=target[split_point:])

In [ ]:
# Enregistrement des resultats

#stats
df_results=pd.DataFrame()

y_true_train = target[:split_point]
predictions_train = target_scaler.inverse_transform(mmm.trace["mu"])
y_true_test = target[split_point:]
predictions_test = target_scaler.inverse_transform(new_predictions)

df_results.at['r2', 'training set'], _ = arviz.r2_score(y_true=y_true_train, y_pred=predictions_train)
df_results.at['r2', 'test set'], _ = arviz.r2_score(y_true=y_true_test, y_pred=predictions_test)
df_results.at['mape', 'training set'] = metrics.mean_absolute_percentage_error(y_true=y_true_train, y_pred=predictions_train.mean(axis=0))
df_results.at['mape', 'test set'] = metrics.mean_absolute_percentage_error(y_true=y_true_test, y_pred=predictions_test.mean(axis=0))
df_results.at['other r2', 'training set'] = r2_score(y_true_train,predictions_train.mean(axis=0))
df_results.at['other r2', 'test set'] = r2_score(y_true_test,predictions_test.mean(axis=0))

update_sheet(spreadsheet_results, 'mmm_stats', df_results)

# predictions
df_results_2 = df[['Date','Ventes']]
df_results_2['predictions'] = np.concatenate([predictions_train.mean(axis=0),predictions_test.mean(axis=0)])
update_sheet(spreadsheet_results, 'predictions', df_results_2)

In [ ]:
# On peut afficher la participation de chaque média au resultats, chaque jour

plot.plot_media_baseline_contribution_area_plot(media_mix_model=mmm,
                                                target_scaler=target_scaler,
                                                fig_size=(30,10))

In [ ]:
# Enregistrement des resultats
contribution_df = plot.create_media_baseline_contribution_df(
      media_mix_model=mmm,
      target_scaler=target_scaler,
      channel_names=networks_list)

update_sheet(spreadsheet_results, 'contribution par jour-semaine', contribution_df)

In [ ]:
# les contributions totales (en %) de chaque média au resultats avec les intervalles de confiance

media_contribution, roi_hat = mmm.get_posterior_metrics(target_scaler=target_scaler, cost_scaler=cost_scaler)

plot.plot_bars_media_metrics(metric=media_contribution, metric_name="Media Contribution Percentage",channel_names=networks_list)

In [ ]:
# Affichage du ROI de chaque média sur la période, avec intervalles de confiance

plot.plot_bars_media_metrics(metric=roi_hat, metric_name="ROI hat",channel_names=networks_list)

In [ ]:
# Enregistrement des resultats

#stats
df_results_3 = pd.DataFrame()

df_results_3['contribution'] = media_contribution.mean(axis=0) # Contribution de chaque network
df_results_3['confidence_int_inf_contribution'] = np.quantile(media_contribution, q=[0.05, 0.95], axis=0)[0,:] # Intervalle de confiance 5% - 95%
df_results_3['confidence_int_sup_contribution'] = np.quantile(media_contribution, q=[0.05, 0.95], axis=0)[1,:]
df_results_3['roi'] = roi_hat.mean(axis=0) # ROI de chaque network
df_results_3['confidence_int_inf_roi'] = np.quantile(roi_hat, q=[0.05, 0.95], axis=0)[0,:] # Intervalle de confiance 5% - 95%
df_results_3['confidence_int_sup_roi'] = np.quantile(roi_hat, q=[0.05, 0.95], axis=0)[1,:]
df_results_3.index=networks_list
update_sheet(spreadsheet_results, 'mmm_results', df_results_3)

In [ ]:
# Affichage des courbes de saturation de chaque média

plot.plot_response_curves(
    media_mix_model=mmm, target_scaler=target_scaler, seed=SEED)

# Attention les KPI des response curves ne sont pas les resultats en € et ne peuvent pas etre interprétés en tant que tel (cf modele complet)

# Code pour améliorer les courbes de saturation

In [ ]:
#Amélioration courbes réponses,

from lightweight_mmm import media_transforms
from lightweight_mmm.plot import _calculate_number_rows_plot
import seaborn as sns
import matplotlib.pyplot as plt
_PALETTE = sns.color_palette(n_colors=100)

#pour un modèle hill_adstock
def media_transform_hill_adstock(media_mix_model,
                                  media_data,
                           lag_weight,
                           half_max_effective_concentration,
                           slope, apply_adstock, normalise):
  """Transforms the input data with the adstock and hill functions.

  Args:
  media_data: Media data to be transformed. It is expected to have 2 dims for
    national models and 3 for geo models.
  normalise: Whether to normalise the output values.

  Returns:
  The transformed media data.
  """
  if media_mix_model.n_geos > 1:
      lag_weight=jnp.repeat(lag_weight,media_mix_model.n_geos).reshape(half_max_effective_concentration.shape)
      slope = jnp.squeeze(jnp.repeat(slope,media_mix_model.n_geos).reshape(half_max_effective_concentration.shape))


  if apply_adstock:
    return media_transforms.hill(
    data=media_transforms.adstock(
        data=media_data, lag_weight=lag_weight, normalise=normalise),
    half_max_effective_concentration=half_max_effective_concentration,
    slope=slope)

  else:
    return media_transforms.hill(
    data=media_data,
    half_max_effective_concentration=half_max_effective_concentration,
    slope=slope)

#pour un modèle carryover
def media_transform_carryover(media_data,
                              ad_effect_retention_rate,
                              peak_effect_delay,
                              exponent,
                              number_lags: int = 13):

  # """Transforms the input data with the adstock and hill functions.

  # Args:
  # media_data: Media data to be transformed. It is expected to have 2 dims for
  #   national models and 3 for geo models.
  # normalise: Whether to normalise the output values.

  # Returns:
  # The transformed media data.
  # """
  # Aujourd'hui on ne fonctionne qu'avec des modèles nationaux
  # if media_mix_model.n_geos > 1:
  #     lag_weight=jnp.repeat(lag_weight,media_mix_model.n_geos).reshape(half_max_effective_concentration.shape)
  #     slope = jnp.squeeze(jnp.repeat(slope,media_mix_model.n_geos).reshape(half_max_effective_concentration.shape))
  # else:
    return media_transforms.apply_exponent_safe(
    data=media_transforms.carryover(data=media_data,
                                    ad_effect_retention_rate=ad_effect_retention_rate,
                                    peak_effect_delay=peak_effect_delay,
                                    number_lags=number_lags),
    exponent=exponent)


In [ ]:
#Test amélioration courbe en couche
def plot_and_df_response_curves(
    percentage_add=2,
    media_mix_model=mmm,
    steps=25,
    costs=costs,
    figure_size=(6,9),
    n_columns=3,
    apply_log_scale=False,
    legend_fontsize = 8,
    media_scaler=media_scaler,
    target_scaler=target_scaler,
    cost_scaler=cost_scaler
  ):

  # if cost_scaler:
  #   costs=cost_scaler.inverse_transform(costs)


  prices=costs/media_data.sum(axis=0)
  media = media_mix_model.media
  media_maxes = media.max(axis=0) * (1 + percentage_add)
  beta_media=media_mix_model.trace['coef_media'].mean(axis=0)

  media_ranges =jnp.linspace(start=0, stop=media_maxes, num=steps)



  if model_name == 'hill_adstock':
    half_max_effective_concentration=media_mix_model._mcmc.get_samples()['half_max_effective_concentration'].mean(axis=0)
    lag_weight=media_mix_model._mcmc.get_samples()['lag_weight'].mean(axis=0)
    slope=media_mix_model._mcmc.get_samples()['slope'].mean(axis=0)

    media_response=beta_media*media_transform_hill_adstock(media_mix_model,
                                                          media_ranges,
                                                          lag_weight,
                                                          half_max_effective_concentration=half_max_effective_concentration,
                                                          apply_adstock=True,
                                                          slope=slope, normalise=True)
  if model_name == 'carryover':
    ad_effect_retention_rate=media_mix_model._mcmc.get_samples()['ad_effect_retention_rate'].mean(axis=0)
    peak_effect_delay=media_mix_model._mcmc.get_samples()['peak_effect_delay'].mean(axis=0)
    exponent=media_mix_model._mcmc.get_samples()['exponent'].mean(axis=0)

    media_response=beta_media*media_transform_carryover(media_ranges,
                                                        ad_effect_retention_rate,
                                                        peak_effect_delay,
                                                        exponent)

  if media_scaler:
    media_ranges=media_scaler.inverse_transform(media_ranges)

  spend_range=media_ranges*prices

  # else:
  #   media_response= TBD (pour adstock)


  if target_scaler:
    media_response = target_scaler.inverse_transform(media_response)

  df=pd.DataFrame(index=[i for i in range(len(media_ranges))])

  for j,k in enumerate(networks_list):
      x=spend_range[:,j]
      z=media_ranges[:,j]
      y=media_response[:,j]

      df[f'{k}_contribution']=y
      df[f'{k}_spend']=x


  fig = plt.figure(media_mix_model.n_media_channels + 1,
                    figsize=figure_size,
                    tight_layout=True)
  n_rows = _calculate_number_rows_plot(
      n_media_channels=media_mix_model.n_media_channels, n_columns=n_columns)
  last_ax = fig.add_subplot(n_rows, 1, n_rows)

  for i in range(media_mix_model.n_media_channels):
    ax = fig.add_subplot(n_rows, n_columns, i + 1)
    sns.lineplot(
        x=spend_range[:, i],
        y=media_response[:, i],
        label=media_mix_model.media_names[i],
        color=_PALETTE[i],
        ax=ax)
    sns.lineplot(
        x=spend_range[:, i],
        y=jnp.log(media_response[:, i]) if apply_log_scale else media_response[:, i],
        label=media_mix_model.media_names[i],
        color=_PALETTE[i],
        ax=last_ax)
    kpi_label='Contribution'
    ax.set_ylabel(kpi_label)
    ax.set_xlabel("Normalized Spend" if not media_scaler else "Spend")
    ax.legend(fontsize=legend_fontsize)

  fig.suptitle("Response curves", fontsize=20)
  last_ax.set_ylabel(kpi_label if not apply_log_scale else f"log({kpi_label})")
  last_ax.set_xlabel("Normalized spend per channel"
                    if not media_scaler else "Spend per channel")
  plt.close()

  return df,fig

# Suite du code

In [ ]:
rc_df, rc_plot=plot_and_df_response_curves(
    percentage_add=2,
    media_mix_model=mmm,
    steps=500,
    costs=costs,
    figure_size=(9,9),
    n_columns=3,
    apply_log_scale=False,
    legend_fontsize = 8,
    media_scaler=media_scaler,
    target_scaler=target_scaler,
    cost_scaler=cost_scaler
    )

In [ ]:
rc_plot

In [ ]:
#Enregistrement des resultats des courbes de saturation
update_sheet(spreadsheet_results, 'saturation_curves', rc_df)

In [ ]:
# Code que l'on ne fait plus tourner (ancienne version)

# #Enregistrement des resultats

# steps = 50  # Nombre d'étapes pour la simulation

# # Reproduire la logique pour obtenir media_maxes, media_ranges et mock_media
# media_maxes = mmm.media.max(axis=0) * 1.2
# media_ranges = jnp.expand_dims(jnp.linspace(start=0, stop=media_maxes, num=steps), axis=0)
# diagonal = jnp.repeat(jnp.eye(mmm.n_media_channels), steps, axis=0).reshape(mmm.n_media_channels, steps, mmm.n_media_channels)
# if mmm.media.ndim == 3:
#     diagonal = jnp.expand_dims(diagonal, axis=-1)
# mock_media = media_ranges * diagonal

# # Utiliser la fonction _make_single_prediction du modèle pour prédire les réponses pour chaque média
# make_single_prediction = jax.vmap(
#     jax.vmap(plot._make_single_prediction, in_axes=(None, 0, None, None), out_axes=0),
#     in_axes=(None, 0, None, None), out_axes=1)

# # Calculer prediction_offset
# prediction_offset = mmm.predict(media=jnp.zeros((1, *mmm.media.shape[1:]))).mean(axis=0)
# if mmm.media.ndim == 3:
#     prediction_offset = jnp.expand_dims(prediction_offset, axis=0)

# # Calculer les prédictions
# predictions = jnp.squeeze(make_single_prediction(mmm, mock_media, None, SEED)) - prediction_offset

# # Inverser la mise à l'échelle si nécessaire
# if target_scaler:
#     predictions = target_scaler.inverse_transform(predictions)
# if media_scaler:
#     media_ranges = media_scaler.inverse_transform(media_ranges)

# # Créer un DataFrame pour les résultats
# media_names = mmm.media_names
# df_dict = {media_names[i]: {'Spend': media_ranges[0,:, i], 'KPI': predictions[:, i]} for i in range(len(media_names))}

# # Créer un DataFrame pour chaque média et les concaténer
# frames = []
# for media_name, data in df_dict.items():
#     media_df = pd.DataFrame({
#         f"{media_name} Spend": data['Spend'].flatten(),  # Aplatir les données si elles sont multidimensionnelles
#         f"{media_name} KPI": data['KPI'].flatten()
#     })
#     frames.append(media_df)

# # Concaténer tous les DataFrames
# saturation_df = pd.concat(frames, axis=1)
# #df_results_3.index=networks_list
# update_sheet(spreadsheet_results, 'saturation_curves', saturation_df)


In [ ]:
###############################
# 5- OPTIMISATION DU BUDGET   #
###############################

## CODE A REVOIR, RESULTATS FAUX A DATE

# Attention: peut etre très long si beaucoup de médias

# Calculer la somme des budgets de chaque canal média
total_budget = jnp.sum(media_data_budget, axis=0)
# Calculer la somme des impressions de chaque canal média
total_impressions = jnp.sum(media_data, axis=0)
# Calculer les ratios entre le budget total du média et les impressions totales du média pour chaque canal média
prices = total_budget / total_impressions
# Pour information, si les impressions ne sont pas renseignées en input, prices sera =1


#budget = jnp.sum(jnp.dot(prices, media_data_budget.mean(axis=0)))* n_time_periods ##On change "media_data" en "media_data_budget"
budget = jnp.sum(jnp.dot(prices, media_data.mean(axis=0)))* n_time_periods #FLORENT

solution, kpi_without_optim, previous_media_allocation = optimize_media.find_optimal_budgets(
    n_time_periods=n_time_periods,
    media_mix_model=mmm,
    budget=budget,
    prices=prices,
    # extra_features=extra_features_scaler.transform(extra_features_test)[:n_time_periods], ##on rajoute les extra_features
    media_scaler=media_scaler,
    target_scaler=target_scaler,
    bounds_lower_pct = bounds_lower_pct,
    bounds_upper_pct = bounds_upper_pct,
    seed=SEED)

In [ ]:
prices

In [ ]:
# Calcul de l'allocation optimale du budget
optimal_buget_allocation = prices * solution.x
previous_budget_allocation = prices * previous_media_allocation

# Enregistrement des resultats
df_optim = pd.DataFrame()
df_optim['previous_budget'] = previous_budget_allocation
df_optim['optimized_budget'] = optimal_buget_allocation
df_optim.index=networks_list
update_sheet(spreadsheet_results, 'budget_optimization', df_optim)

df_optim_2 = pd.DataFrame()
df_optim_2.at['results', 'before optimization'] = -kpi_without_optim
df_optim_2.at['results', 'after optimization'] = -solution['fun']
update_sheet(spreadsheet_results, 'results_optimization', df_optim_2)


In [ ]:
# Affichage des resultats de l'optimisation
plot.plot_pre_post_budget_allocation_comparison(media_mix_model=mmm,
                                                kpi_with_optim=solution['fun'],
                                                kpi_without_optim=kpi_without_optim,
                                                optimal_buget_allocation=optimal_buget_allocation,
                                                previous_budget_allocation=previous_budget_allocation,
                                                figure_size=(10,10))

In [ ]:
# INTEGRATION

#Sauvegarde du modele
file_path = "media_mix_model.pkl"
utils.save_model(media_mix_model=mmm, file_path=file_path)

# Chargement du modèle
loaded_mmm = utils.load_model(file_path=file_path)